## Imports


In [ ]:
# Source for some of unet code which was adapted into this notebook: https://github.com/zhixuhao/unet

from __future__ import print_function

import numpy as np
import pandas as pd
import os
import glob

import skimage.io as io
from skimage.io import imshow, imread
import skimage.transform as trans
from skimage.transform import rotate, resize
import matplotlib.pyplot as plt
from random import randint, choice

from keras.preprocessing.image import ImageDataGenerator
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, History
from keras import backend as keras
from keras import backend as K
import tensorflow as tf

## Set parameters and directories

In [ ]:
# parameters
EXPERIMENT_ID = "01"
BATCHSIZE_TRAIN = 8 # batch size of 8 used in our publication
BATCHSIZE_TEST = 1 
THRESHOLD = 0.5
NUM_EPOCHS = 55

# directories
INPUT_DIR = "" # directory containing images and masks for training / testing Unet
SAVE_DIR = "" # directory for saving Unet-predicted image masks for test dataset, and other output files

# load dataframes from train test splitting Excel file (update file path accordingly)
TRAIN_DF = pd.read_excel("Unet_train_test.xlsx", sheet_name="training", dtype=str) 
TEST_DF = pd.read_excel("Unet_train_test.xlsx", sheet_name="test", dtype=str)

os.chdir(INPUT_DIR)

## Adapting Keras for unet segmentation

In [ ]:
def adjustData(img,mask,num_class):
    img = img / 255
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img,mask)

def rand_contrast_gradient (img):
    image = img
    if image.ndim == 2:
        height, width = image.shape
        gradient = np.ones((height, width))
    if image.ndim == 3:
        height, width, depth = image.shape
        gradient = np.ones((height, width, depth))
        
    random_grad = choice([1.2, 1.3, 1.5, 2, 4, 5, 10, 20, 500, 500, 500, 500, 500])
    for i in range(height*2):
        gradient[i:] -= 1/(height*random_grad)
        
    random_angle = randint(0,360)
    gradient = rotate(gradient, random_angle, resize=False, mode='edge')
    
    modified_image = (image*gradient).astype('float32')
    return (modified_image)

def theGenerator(batch_size,dataframe,directory,aug_dict_image,aug_dict_mask,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix = "image",mask_save_prefix = "mask",
                    num_class = 2,save_to_dir = None,target_size = (512,512),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict_image)
    mask_datagen = ImageDataGenerator(**aug_dict_mask)
    image_generator = image_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = directory,
        x_col = 'filenames',
        y_col = None,
        classes = None,
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = directory,
        x_col = 'filenames',
        y_col = None,
        classes = None,
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    the_generator = zip(image_generator, mask_generator)
    for (img,mask) in the_generator:
        img,mask = adjustData(img,mask,num_class)
        yield (img,mask)

def saveResult(dataframe,directory,npyfile,x_col = 'filenames', num_class = 2):
    test_image_filenames = dataframe[x_col]
    for filename, npimage in zip(test_image_filenames, npyfile):
        img = npimage[:,:,0]
        img_thresh = (img > THRESHOLD).astype(np.float32)
        io.imsave(os.path.join(directory,f"{filename}_PRED_THRESH.png"), img_thresh)

# Two functions below are from: https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2#--responses        
        
def iou_coef(y_true, y_pred, smooth=1):
    y_true_ = tf.dtypes.cast(y_true, tf.int32)
    y_pred_ = tf.dtypes.cast(y_pred > 0.5, tf.int32)
    intersection = K.sum(K.abs(y_true_ * y_pred_), axis=[1,2,3])
    union = K.sum(y_true_, [1,2,3]) + K.sum(y_pred_, [1,2,3]) - intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_coef(y_true, y_pred, smooth=1):
    y_true_ = tf.dtypes.cast(y_true, tf.int32)
    y_pred_ = tf.dtypes.cast(y_pred > 0.5, tf.int32)
    intersection = K.sum(y_true_ * y_pred_, axis=[1,2,3])
    union = K.sum(y_true_, [1,2,3]) + K.sum(y_pred_, [1,2,3])
    dice = K.mean((2 * intersection + smooth) / (union + smooth), axis=0)
    return dice

## Data augmentation

In [ ]:
#if you don't want to do data augmentation, set data_gen_args as an empty dict. e.g. data_gen_args = dict()

train_data_gen_args_image = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest',
                    preprocessing_function = rand_contrast_gradient)

train_data_gen_args_mask = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

test_data_gen_args_image = dict()
test_data_gen_args_mask = dict()

myGeneTrain = theGenerator(BATCHSIZE_TRAIN, TRAIN_DF, INPUT_DIR, train_data_gen_args_image, train_data_gen_args_mask)
myGeneTest = theGenerator(BATCHSIZE_TEST, TEST_DF, INPUT_DIR, test_data_gen_args_image, test_data_gen_args_mask)

STEP_SIZE_TRAIN=len(TRAIN_DF.index)//BATCHSIZE_TRAIN
STEP_SIZE_TEST=len(TEST_DF.index)//BATCHSIZE_TEST

print(STEP_SIZE_TRAIN)
print(STEP_SIZE_TEST)

## Building unet

In [ ]:
def unet(pretrained_weights = None,input_size = (512,512,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)
    
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy', iou_coef, dice_coef])
    
    return model

## Training Unet

In [ ]:
model = unet()
mc = ModelCheckpoint(f"{SAVE_DIR}/unet_implant_{EXPERIMENT_ID}_BESTIOU.hdf5", 
                                   monitor='iou_coef',
                                   verbose=1, 
                                   save_best_only=True, 
                                   mode='max')
mc2 = ModelCheckpoint(f"{SAVE_DIR}/unet_implant_{EXPERIMENT_ID}_FINALEPOCH.hdf5", 
                     monitor=save_best_metric, mode='auto', verbose=1, save_best_only=False, period=5)   # to save last epoch
history = model.fit_generator(generator=myGeneTrain,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=myGeneTest,
                              validation_steps=STEP_SIZE_TEST,
                              epochs=NUM_EPOCHS,
                              callbacks=[mc, mc2])

In [ ]:
# Plot training accuracy values
plt.plot(history.history['accuracy'], color='k', linestyle='-')
plt.plot(history.history['val_accuracy'], color='r', linestyle='-')
plt.title('Model accuracy',  color='k')
plt.ylabel('Accuracy',  color='k')
plt.xlabel('Epoch',  color='k')
plt.legend(['Training', 'Test'], loc='upper left')
plt.tick_params(colors='k')
plt.xlim(0, NUM_EPOCHS)
plt.ylim(top=1)
plt.savefig(f"{SAVE_DIR}/unet_accuracy_curve_{EXPERIMENT_ID}.png", dpi=300, facecolor='w', edgecolor='w')
plt.show()


# Plot training loss values
plt.plot(history.history['loss'], color='k', linestyle='-')
plt.plot(history.history['val_loss'], color='r', linestyle='-')
plt.title('Model loss', color='k')
plt.ylabel('Loss', color='k')
plt.xlabel('Epoch', color='k')
plt.legend(['Training', 'Test'], loc='upper right')
plt.tick_params(colors='k')
plt.xlim(0, NUM_EPOCHS)
plt.ylim(bottom=0)
plt.savefig(f"{SAVE_DIR}/unet_loss_curve_{EXPERIMENT_ID}.png", dpi=300, facecolor='w', edgecolor='w')
plt.show()


# Plot IOU and Dice coefficient values
plt.plot(history.history['iou_coef'], color='k', linestyle='-')
plt.plot(history.history['dice_coef'], color='k', linestyle='--')
plt.plot(history.history['val_iou_coef'], color='r', linestyle='-')
plt.plot(history.history['val_dice_coef'], color='r', linestyle='--')
plt.title('Model intersection-over-union and Dice coefficient',  color='k')
plt.ylabel('Coefficient value',  color='k')
plt.xlabel('Epoch',  color='k')
plt.legend(['Train IOU', 'Train Dice', 'Test IOU', 'Test Dice'], loc='upper left')
plt.tick_params(colors='k')
plt.xlim(0, NUM_EPOCHS)
plt.ylim(0, 1)
plt.savefig(f"{SAVE_DIR}/unet_iou_dice_curve_{EXPERIMENT_ID}.png", dpi=300, facecolor='w', edgecolor='w')
plt.show()


## Save test dataset unet-predicted masks and display them

In [ ]:
model = unet()
model.load_weights(f"{SAVE_DIR}/unet_implant_{EXPERIMENT_ID}_FINALEPOCH.hdf5")
results = model.predict_generator(myGeneTest,STEP_SIZE_TEST,verbose=1)
saveResult(TEST_DF, SAVE_DIR, results)

# Parameter counts (source: https://stackoverflow.com/questions/45046525/how-can-i-get-the-number-of-trainable-parameters-of-a-model-in-keras)

trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])

print('\nTotal params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))

In [ ]:
predictions = [png for png in os.listdir(SAVE_DIR) if png[-10:]=="THRESH.png"]

fig, ax = plt.subplots(nrows=STEP_SIZE_TEST, ncols=1, figsize=(20,STEP_SIZE_TEST*15))

for index, prediction in enumerate(predictions):
    ds = io.imread(f"{SAVE_DIR}/{prediction}")
    ax[index].imshow(ds, cmap='gray')
    ax[index].set_title(f"FILE: {prediction}", fontsize=10, color='w')
    ax[index].axis('off')
       
plt.show()